In [123]:
import re
import pandas as pd
import numpy as np

In [124]:
df = pd.read_csv('input/attacks.csv', encoding='cp1252')

In [125]:
df.sample(10)

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
24637,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20169,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11766,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22632,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22917,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9097,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19611,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22470,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22675,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [126]:
df.drop(["Year", "Time", "Species ", 
         "Investigator or Source", "pdf", 
         "href formula", "href", "Case Number.1", 
         "Case Number.2", "original order", "Unnamed: 22", 
         "Unnamed: 23"], axis=1, inplace=True)

In [127]:
df.sample(10)

,Case Number,Date,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N)
13532,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19595,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15070,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21876,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20854,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
798,2012.04.19.b,19-Apr-2012,Unprovoked,USA,Florida,"Indialantic, Brevard County",Surfing,Justin Ellingham,M,28,Lacerations to hand,N
9252,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20679,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13635,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22113,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [128]:
df = df[~df.isin([0, np.NAN, '', "NaN", "xx", "0"]).all(1)]

In [129]:
df.sample(10)

,Case Number,Date,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N)
2085,1999.12.02,02-Dec-1999,Unprovoked,USA,Florida,"Boynton Beach, Palm Beach County",Surfing,male,M,35,Right arm & fingers lacerated,N
5276,1919.00.00,1919,Unprovoked,USA,Florida,"Near Panama City, Bay County",Swimming,adult female (Mrs. Avery?),F,NaN,Leg severed,N
4427,1951.11.29,29-Nov-1951,Unprovoked,SOUTH AFRICA,KwaZulu-Natal,"South Beach, Durban",Swimming,Harold Tait,M,23,Thigh bruised & abraded,N
2633,1990.05.13,13-May-1990,Provoked,SOUTH AFRICA,KwaZulu-Natal,Protea Reef,Fishing,"skiboat Double One, occupants: Anton & Michell...",NaN,NaN,"No injury to occupants, hooked shark freed its...",N
3541,1966.12.27.b,27-Dec-1966,Provoked,AUSTRALIA,New South Wales,South West Rocks,Spearfishing,Robert Lusted,M,29,The shark bit the diver's leg after he shot it...,N
4600,1946.04.19,19-Apr-1946,Unprovoked,AUSTRALIA,Queensland,"Trinity Beach, Cairns",Swimming,Robert McAuliffe,M,17,FATAL,Y
3736,1963.10.15,15-Oct-1963,Provoked,NORTH PACIFIC OCEAN,NaN,NaN,Fishing for sharks,"Eichi Nakata, of the the crew of the trawler K...",M,20,"Right hand lacerated, PROVOKED INCIDENT",N
5721,1888.12.25.R,Reported 25-Dec-1888,Unprovoked,INDIAN OCEAN,Between Perth & Colombo,NaN,Swimming,E. Burnside,M,NaN,Foot severed,N
927,2011.05.04,04-May-2011,Unprovoked,SOUTH AFRICA,KwaZulu-Natal,Palm Beach,Spearfishing,Trevor Burger,M,37,Calf bitten,N
279,2016.05.03,03-May-2016,Unprovoked,USA,Hawaii,"Wailea Beach, Maui",Floating,male,M,59,Minor lacerations to right shoulder,N


In [130]:
df = df.drop_duplicates()
df

,Case Number,Date,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N)
0,2018.06.25,25-Jun-2018,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N
1,2018.06.18,18-Jun-2018,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N
2,2018.06.09,09-Jun-2018,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N
3,2018.06.08,08-Jun-2018,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N
4,2018.06.04,04-Jun-2018,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N
...,...,...,...,...,...,...,...,...,...,...,...,...
6297,ND.0005,Before 1903,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,male,M,NaN,FATAL,Y
6298,ND.0004,Before 1903,Unprovoked,AUSTRALIA,Western Australia,NaN,Pearl diving,Ahmun,M,NaN,FATAL,Y
6299,ND.0003,1900-1905,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,M,NaN,FATAL,Y
6300,ND.0002,1883-1889,Unprovoked,PANAMA,NaN,"Panama Bay 8ºN, 79ºW",NaN,Jules Patterson,M,NaN,FATAL,Y


In [131]:
counter = 0
df2 = pd.DataFrame()
indexes = df.index.values.tolist()
for i in range(len(indexes)-1):
    data = df.iloc[[indexes[i]]]
    for column in df.columns:
            if df.at[indexes[i], column] == df.at[indexes[i+1], column]:
                counter += 1
    if counter < 9:
        df2 = pd.concat([df2, data])
    counter = 0
df = df2

In [132]:
df

,Case Number,Date,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N)
0,2018.06.25,25-Jun-2018,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N
1,2018.06.18,18-Jun-2018,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N
2,2018.06.09,09-Jun-2018,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N
3,2018.06.08,08-Jun-2018,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N
4,2018.06.04,04-Jun-2018,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N
...,...,...,...,...,...,...,...,...,...,...,...,...
6297,ND.0005,Before 1903,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,male,M,NaN,FATAL,Y
6298,ND.0004,Before 1903,Unprovoked,AUSTRALIA,Western Australia,NaN,Pearl diving,Ahmun,M,NaN,FATAL,Y
6299,ND.0003,1900-1905,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,M,NaN,FATAL,Y
6300,ND.0002,1883-1889,Unprovoked,PANAMA,NaN,"Panama Bay 8ºN, 79ºW",NaN,Jules Patterson,M,NaN,FATAL,Y


In [133]:
def surfing(x):
    if re.search("[sS]urf", str(x)):
        return "Surfing"
    else:
        return x

df["Activity"] = df["Activity"].apply(surfing)

In [134]:
def leg(x):
    if re.search("\s?([lL][eE][gG][sS]?|[fF][eEoO][eEoO][tT]?|[tT][hH][iI][gG][hH][sS]?|[aA][nN][kK][lL][eE][sS]?)\s?", str(x)):
        return "Y"
    else:
        return "N"
df["Leg Injured"] = df["Injury"].apply(leg)

In [135]:
df

,Case Number,Date,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Leg Injured
0,2018.06.25,25-Jun-2018,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,N
1,2018.06.18,18-Jun-2018,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,Y
2,2018.06.09,09-Jun-2018,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,Y
3,2018.06.08,08-Jun-2018,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,Y
4,2018.06.04,04-Jun-2018,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6297,ND.0005,Before 1903,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,male,M,NaN,FATAL,Y,N
6298,ND.0004,Before 1903,Unprovoked,AUSTRALIA,Western Australia,NaN,Pearl diving,Ahmun,M,NaN,FATAL,Y,N
6299,ND.0003,1900-1905,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,M,NaN,FATAL,Y,N
6300,ND.0002,1883-1889,Unprovoked,PANAMA,NaN,"Panama Bay 8ºN, 79ºW",NaN,Jules Patterson,M,NaN,FATAL,Y,N


In [136]:
df = df[(df['Injury'].notna()) & (df['Activity'].notna())]

In [137]:
df = df.rename(columns={'Sex ': 'Sex'})

In [138]:
df.drop(df[(df["Sex"]=="lli") | (df["Sex"]=="N") | (df["Sex"]==".")].index, inplace = True)

In [139]:
def male(x):
    if re.search("M\s", str(x)):
        return "M"
    else:
        return x

df["Sex"] = df["Sex"].apply(male)

In [140]:
df["Sex"].unique()

array(['F', 'M', nan], dtype=object)

In [141]:
df.to_csv("output/cleaned_data.csv")